# Midterm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options

sns.set_style("whitegrid")
sns.set_context("paper")

In [ ]:
output_path = Path() / "midterm_output"
output_path.mkdir(parents=True, exist_ok=True)

service = QiskitRuntimeService(channel="ibm_quantum")
backend_sim = service.get_backend("ibmq_qasm_simulator")
backend_qc = service.least_busy(simulator=False)

# options_sim = Options()
# options_sim.simulator.seed_simulator = 42
# options_sim.execution.shots = 2000
# options_sim.optimization_level = 0 # no optimization
# options_sim.resilience_level = 0 # no error mitigation

In [ ]:
n_bits = 3
numbers = np.arange(2 ** n_bits)
for i in numbers:
    j = np.binary_repr(i, width=n_bits)
    print(f"{i} ---> {j}")

## Step 1

In [ ]:
def increment_1_circuit(i):
    j = np.binary_repr(i, width=3)
    sv = Statevector.from_label(j).data
    circuit = QuantumCircuit(3)
    circuit.initialize(sv, circuit.qubits)
    circuit.cnot(0, 1)
    circuit.x(1)
    circuit.ccx(0, 1, 2)
    circuit.x([0, 1])
    circuit.measure_all()
    return circuit

In [ ]:
circuit_example_1 = increment_1_circuit(1)
circuit_example_1.draw("mpl", plot_barriers=False, filename=None)#output_path / "circuit.png")

In [ ]:
# Simulator
with Session(service=service, backend=backend_sim) as session_sim:
    sampler = Sampler(session=session_sim)
    job = sampler.run([increment_1_circuit(i) for i in numbers] )
    result = job.result()
dict_1_sim = {i: max(d, key=d.get) for i, d in zip(numbers, result.quasi_dists)}
dict_1_sim

In [ ]:
# Quantum Computer
with Session(service=service, backend=backend_qc) as session_qc:
    sampler = Sampler(session=session_qc)
    job = sampler.run([increment_1_circuit(i) for i in numbers] )
    result = job.result()
dict_1_qc = {i: max(d, key=d.get) for i, d in zip(numbers, result.quasi_dists)}
dict_1_qc

## Step 2

In [ ]:
def increment_3_circuit(i):
    j = np.binary_repr(i, width=3)
    sv = Statevector.from_label(j).data
    circuit = QuantumCircuit(3)
    circuit.initialize(sv, circuit.qubits)
    circuit.cnot(0, 1)
    circuit.x(1)
    circuit.ccx(0, 1, 2)
    circuit.x(0)
    circuit.cnot(0, 1)
    circuit.ccx(0, 1, 2)
    circuit.x(0)
    circuit.cnot(0, 1)
    circuit.ccx(0, 1, 2)
    circuit.x([0, 1])
    circuit.measure_all()
    return circuit

In [ ]:
circuit_example_3 = increment_3_circuit(1)
circuit_example_3.draw("mpl", plot_barriers=False, filename=None)#output_path / "circuit.png")

In [ ]:
# Simulator
with Session(service=service, backend=backend_sim) as session_sim:
    sampler = Sampler(session=session_sim)
    job = sampler.run([increment_3_circuit(i) for i in numbers] )
    result = job.result()
dict_3_sim = {i: max(d, key=d.get) for i, d in zip(numbers, result.quasi_dists)}
dict_3_sim

In [ ]:
# Quantum Computer
with Session(service=service, backend=backend_qc) as session_qc:
    sampler = Sampler(session=session_qc)
    job = sampler.run([increment_3_circuit(i) for i in numbers] )
    result = job.result()
dict_3_qc = {i: max(d, key=d.get) for i, d in zip(numbers, result.quasi_dists)}
dict_3_qc